In [1]:
import numpy as np 
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier

In [2]:
print ("Get tables to combine")
content = pd.read_csv('all/promoted_content.csv')
print('Done combining')

chunksize=50000
train = pd.read_csv('all/clicks_train.csv',iterator=True,chunksize=chunksize)
print('Training')
for chunk in train:
    chunk=pd.merge(chunk,content,how='left',on='ad_id')	
    predictors=[x for x in chunk.columns if x not in ['display_id','clicked']]
    #print(predictors) #['ad_id', 'document_id', 'campaign_id', 'advertiser_id']
    chunk=chunk.fillna(0.0)
    alg = RandomForestClassifier(random_state=1, n_estimators=10, min_samples_split=4, min_samples_leaf=2, warm_start=True)
    alg.fit(chunk[predictors], chunk["clicked"])
print('5')


Get tables to combine
Done combining
Training
5


In [3]:
train=''
print('Testing')
test= pd.read_csv('all/clicks_test.csv',iterator=True,chunksize=chunksize) #Load data
predY=[]
for chunk in test:
    init_chunk_size=len(chunk)
    chunk=pd.merge(chunk,content,how='left',on='ad_id')
    chunk=chunk.fillna(0.0)
    chunk_pred=list(alg.predict_proba(chunk[predictors]).astype(float)[:,1])
    predY += chunk_pred
print('Done Testing')

Testing
Done Testing


In [4]:
print('Preparing for Submission')
test=''
test= pd.read_csv('all/clicks_test.csv')
results=pd.concat((test,pd.DataFrame(predY)) ,axis=1,ignore_index=True)
print(results.head(10))
results.columns = ['display_id','ad_id','clicked']
results.sort_values(['display_id','clicked'], inplace=True, ascending=False)
results = results.groupby('display_id').ad_id.apply(lambda x: " ".join(map(str,x))).reset_index()
results.to_csv('all/final2_3dec.csv', index=False)
print("output stored")

Preparing for Submission
          0       1         2
0  16874594   66758  0.120000
1  16874594  150083  0.104339
2  16874594  162754  0.195243
3  16874594  170392  0.184841
4  16874594  172888  0.518810
5  16874594  180797  0.286017
6  16874595    8846  0.142857
7  16874595   30609  0.000000
8  16874595  143982  0.000000
9  16874596   11430  0.370992
output stored
